"""
core3_02_rulegrid_simulator.ipynb

목적:
- Core 2의 prediction-trigger 구조를 유지한 채
- 규칙(policy)을 강화/변형한 여러 variant를 적용해
- decision instability가 줄어드는지/오히려 악화되는지를 관찰한다.

중요:
- 예측 성능 비교 ❌
- cutoff ❌
- rule variant별 decision dynamics(trace)만 기록
"""

In [1]:
import json
import random
import pandas as pd
import numpy as np
from pathlib import Path

random.seed(42)

# Core 2 artifact
CORE2_DIR = Path("../artifact/core2")

prediction_trace_base = pd.read_csv(
    CORE2_DIR / "prediction_trace.csv"
)

mutation_candidates = pd.read_csv(
    CORE2_DIR / "mutation_candidate_set.csv"
)

prediction_trace_base.head()

TARGET_ANTIBODIES = [
    "GDPa1-001",  # abagovomab
    "GDPa1-045",  # cixutumumab
    "GDPa1-183",  # prolgolimab
]

prediction_trace_base = prediction_trace_base[
    prediction_trace_base["antibody_key"].isin(TARGET_ANTIBODIES)
].reset_index(drop=True)

prediction_trace_base[["antibody_key"]].drop_duplicates() # 대상 항체 고정 (Core 1 기준)

,antibody_key
0,GDPa1-001
20,GDPa1-045
40,GDPa1-183


Rule Variant Grid 정의 (핵심)

여기서 규칙만 바뀐다
예측 구조 / mutation candidate는 그대로

In [2]:
# 1) multi-objective gate 방식
MULTI_OBJECTIVE_GATES = [
    "mean_only",          # 단순 평균
    "mean_plus_std",      # 평균 + 변동성
    "worst_of_three"      # 최악 지표 기준
]

# 2) mutation strength 제한
MUTATION_STRENGTHS = [
    "none",        # 제한 없음
    "weak_only",   # 약한 mutation만 허용
    "single_step"  # step당 1회만 허용
]

# 3) hysteresis / cooldown
COOLDOWN_VARIANTS = [
    0,   # 없음
    1,   # 1-step cooldown
    3    # 3-step cooldown
]

# Cartesian product
rule_grid = []

variant_id = 0
for gate in MULTI_OBJECTIVE_GATES:
    for strength in MUTATION_STRENGTHS:
        for cooldown in COOLDOWN_VARIANTS:
            rule_grid.append({
                "variant_id": f"rule_v{variant_id:02d}",
                "gate": gate,
                "mutation_strength": strength,
                "cooldown": cooldown
            })
            variant_id += 1

len(rule_grid), rule_grid[:3]

(27,
 [{'variant_id': 'rule_v00',
   'gate': 'mean_only',
   'mutation_strength': 'none',
   'cooldown': 0},
  {'variant_id': 'rule_v01',
   'gate': 'mean_only',
   'mutation_strength': 'none',
   'cooldown': 1},
  {'variant_id': 'rule_v02',
   'gate': 'mean_only',
   'mutation_strength': 'none',
   'cooldown': 3}])

 Risk score 계산 함수 (gate별)

❗ cutoff 없음
❗ 연속값 유지

In [3]:
RISK_COLUMNS = [
    "stability_score",
    "aggregation_score",
    "solubility_score"
]

def compute_risk_by_gate(row, gate):
    values = row[RISK_COLUMNS].values
    
    if gate == "mean_only":
        return np.mean(values)
    
    if gate == "mean_plus_std":
        return np.mean(values) + np.std(values)
    
    if gate == "worst_of_three":
        return np.max(values)
    
    raise ValueError(f"Unknown gate: {gate}")

단일 variant 시뮬레이션 함수

Core 2 구조 그대로
규칙만 다르게

In [4]:
def run_rule_variant_simulation(
    variant,
    base_trace,
    mutation_candidates,
    max_steps=20
):
    records = []
    
    for ab in TARGET_ANTIBODIES:
        ab_trace = base_trace[
            base_trace["antibody_key"] == ab
        ].sort_values("step").reset_index(drop=True)
        
        prev_score = ab_trace.loc[0, "pred_score"]
        current_sequence = ab_trace.loc[0, "sequence_current"]
        intervention_count = 0
        cooldown_left = 0
        
        for step in range(1, max_steps + 1):
            # 예측값은 Core 2처럼 흔들리게 유지
            noise = random.uniform(-0.05, 0.05)
            current_score = prev_score + noise
            delta = current_score - prev_score
            
            decision = "HOLD"
            
            if cooldown_left > 0:
                cooldown_left -= 1
            else:
                if delta > 0:
                    decision = "MUTATE"
            
            mutation_id_applied = None
            
            if decision == "MUTATE":
                candidates = mutation_candidates[
                    (mutation_candidates["antibody_key"] == ab) &
                    (mutation_candidates["step"] == step)
                ]
                
                if len(candidates) > 0:
                    chosen = candidates.sample(1).iloc[0]
                    current_sequence = chosen["sequence_after"]
                    mutation_id_applied = chosen["mutation_id"]
                    intervention_count += 1
                    
                    if variant["cooldown"] > 0:
                        cooldown_left = variant["cooldown"]
            
            records.append({
                "variant_id": variant["variant_id"],
                "antibody_key": ab,
                "step": step,
                "pred_score": current_score,
                "pred_score_delta": delta,
                "decision": decision,
                "mutation_id_applied": mutation_id_applied,
                "intervention_count_cum": intervention_count,
                "gate": variant["gate"],
                "mutation_strength": variant["mutation_strength"],
                "cooldown": variant["cooldown"]
            })
            
            prev_score = current_score
    
    return pd.DataFrame(records)

In [5]:
all_variant_traces = []

for variant in rule_grid:
    df_variant = run_rule_variant_simulation(
        variant=variant,
        base_trace=prediction_trace_base,
        mutation_candidates=mutation_candidates,
        max_steps=20
    )
    all_variant_traces.append(df_variant)

prediction_trace_rulegrid = pd.concat(
    all_variant_traces,
    ignore_index=True
)

prediction_trace_rulegrid.head() # Rule Grid 전체 실행

,variant_id,antibody_key,step,pred_score,pred_score_delta,decision,mutation_id_applied,intervention_count_cum,gate,mutation_strength,cooldown
0,rule_v00,GDPa1-001,1,2.027885,0.013943,MUTATE,GDPa1-001_mut1_1,1,mean_only,none,0
1,rule_v00,GDPa1-001,2,1.980386,-0.047499,HOLD,None,1,mean_only,none,0
2,rule_v00,GDPa1-001,3,1.957889,-0.022497,HOLD,None,1,mean_only,none,0
3,rule_v00,GDPa1-001,4,1.930210,-0.027679,HOLD,None,1,mean_only,none,0
4,rule_v00,GDPa1-001,5,1.953858,0.023647,MUTATE,GDPa1-001_mut0_5,2,mean_only,none,0


In [7]:
print(prediction_trace_rulegrid.columns.tolist())
print("Total rows:", len(prediction_trace_rulegrid)) # 결과 스키마 확인

['variant_id', 'antibody_key', 'step', 'pred_score', 'pred_score_delta', 'decision', 'mutation_id_applied', 'intervention_count_cum', 'gate', 'mutation_strength', 'cooldown']
Total rows: 1620


In [6]:
OUTPUT_DIR = Path("../artifact/core3")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / "prediction_trace_rulegrid.csv"
prediction_trace_rulegrid.to_csv(output_path, index=False)

output_path

PosixPath('../artifact/core3/prediction_trace_rulegrid.csv')